In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from datasets import load_dataset, concatenate_datasets
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_from_disk
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
import torch
import numpy as np
from pcst_fast import pcst_fast
from torch_geometric.data.data import Data

/vol/hd1/users/ahmadi/sadaf/GraphNeighborLM/sadafenv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset = load_dataset("rmanluo/RoG-webqsp")

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
        num_rows: 2826
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
        num_rows: 246
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
        num_rows: 1628
    })
})

In [10]:
dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])
dataset

Dataset({
    features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
    num_rows: 4700
})

In [11]:
seed = 0
percent_data = 0.02
# 10% of the dataset to be reserved as the main dataset for this thesis
train_test_split = dataset.train_test_split(test_size=percent_data, seed=seed)

X_train = train_test_split['train']
dataset = train_test_split['test']

In [12]:

dataset

Dataset({
    features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
    num_rows: 94
})

In [13]:
# تعداد داده‌های موردنظر
train_size = 47
val_size = 20
test_size = 27

# انتخاب داده‌ها
train_data = dataset.select(range(train_size))
val_data = dataset.select(range(train_size, train_size + val_size))
test_data = dataset.select(range(train_size + val_size, train_size + val_size + test_size))


# Combine the splits into a DatasetDict
sample_dataset = DatasetDict({
    "train": train_data,
    "validation": val_data,
    "test":test_data
})


In [14]:
sample_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
        num_rows: 47
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
        num_rows: 20
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'q_entity', 'a_entity', 'graph', 'choices'],
        num_rows: 27
    })
})

In [15]:
model_name = 'sbert'
path = '/home/ahmadi/sadaf/GraphNeighborLM/G-retriever/dataset/WebQuestionSP'
path_nodes = f'{path}/nodes'
path_edges = f'{path}/edges'
path_graphs = f'{path}/graphs'

In [16]:
from datasets import DatasetDict

sample_dataset.save_to_disk(f'{path}/processed_dataset')


Saving the dataset (1/1 shards): 100%|██████████| 27/27 [00:00<00:00, 1239.30 examples/s]
